# Algorithm 1: Generalized ID Algorithm for ioSCMs

**Source:** Forré & Mooij (2019) - "Causal Calculus in the Presence of Cycles, Latent Confounders and Selection Bias"

---

## Algorithm Pseudocode

### Function: ID (Main Function)

$$
\begin{align*}
&\textbf{1: function } \mathbf{ID}(G, \mathbf{Y}, \mathbf{W}, P(\mathbf{V} | do(\mathbf{J}))) \\
&\textbf{2: require: } \mathbf{Y} \subseteq \mathbf{V}, \mathbf{W} \subseteq \mathbf{V}, \mathbf{Y} \cap \mathbf{W} = \emptyset \\
&\textbf{3: } H \leftarrow \text{Anc}^{G_{\mathbf{V} \setminus \mathbf{W}}}(\mathbf{Y}) \\
&\textbf{4: } \textbf{for } C \in \text{CD}(H) \textbf{ do} \\
&\textbf{5: } \quad Q[C] \leftarrow \text{IDCD}(G, C, \text{Cd}^G(C), Q[\text{Cd}^G(C)]) \\
&\textbf{6: } \quad \textbf{if } Q[C] = \text{FAIL} \textbf{ then} \\
&\textbf{7: } \quad\quad \textbf{return } \text{FAIL} \\
&\textbf{8: } \quad \textbf{end if} \\
&\textbf{9: } \textbf{end for} \\
&\textbf{10: } Q[H] \leftarrow \left[ \bigotimes_{C \in \text{CD}(H)} \right] Q[C] \\
&\textbf{11: } \textbf{return } P(\mathbf{Y} | do(\mathbf{J}, \mathbf{W})) = \int Q[H] \, d\mathbf{x}_{H \setminus \mathbf{Y}} \\
&\textbf{12: } \textbf{end function}
\end{align*}
$$

---

### Function: IDCD (Recursive Helper Function)

$$
\begin{align*}
&\textbf{13: function } \mathbf{IDCD}(G, \mathbf{C}, \mathbf{D}, Q[\mathbf{D}]) \\
&\textbf{14: require: } \mathbf{C} \subseteq \mathbf{D} \subseteq \mathbf{V}, \text{CD}(G_{\mathbf{D}}) = \{\mathbf{D}\} \\
&\textbf{15: } A \leftarrow \text{Anc}^{G[\mathbf{D}]}(\mathbf{C}) \cap \mathbf{D} \\
&\textbf{16: } Q[A] \leftarrow \int Q[\mathbf{D}] \, d(\mathbf{x}_{\mathbf{D} \setminus A}) \\
&\textbf{17: } \textbf{if } A = \mathbf{C} \textbf{ then} \\
&\textbf{18: } \quad \textbf{return } Q[A] \\
&\textbf{19: } \textbf{else if } A = \mathbf{D} \textbf{ then} \\
&\textbf{20: } \quad \textbf{return } \text{FAIL} \\
&\textbf{21: } \textbf{else if } \mathbf{C} \subset A \subset \mathbf{D} \textbf{ then} \\
&\textbf{22: } \quad \textbf{for } S \in \mathcal{S}(G[A]) \text{ s.t. } S \subseteq \text{Cd}^{G[A]}(\mathbf{C}) \textbf{ do} \\
&\textbf{23: } \quad\quad R_A[S] \leftarrow P(S | \text{Pred}^G_<(S) \cap A, do(\mathbf{J} \cup \mathbf{V} \setminus A)) \\
&\textbf{24: } \quad \textbf{end for} \\
&\textbf{25: } \quad Q[\text{Cd}^{G[A]}(\mathbf{C})] \leftarrow \bigotimes_{\substack{S \in \mathcal{S}(G[A]) \\ S \subseteq \text{Cd}^{G[A]}(\mathbf{C})}} R_A[S] \\
&\textbf{26: } \quad \textbf{return } \text{IDCD}(G, \mathbf{C}, \text{Cd}^{G[A]}(\mathbf{C}), Q[\text{Cd}^{G[A]}(\mathbf{C})]) \\
&\textbf{27: } \textbf{end if} \\
&\textbf{28: } \textbf{end function}
\end{align*}
$$

### Line 1: Function Declaration 

$$
\textbf{1: function } \mathbf{ID}(G, \mathbf{Y}, \mathbf{W}, P(\mathbf{V} | do(\mathbf{J})))
$$

---

#### 1. Symbols

| Symbol | Meaning |
|--------|---------|
| $\mathbf{ID}$ | Identification algorithm function |
| $G$ | Directed mixed graph (DMG) with possible cycles |
| $\mathbf{Y}$ | Target/outcome variables (what we want to predict) |
| $\mathbf{W}$ | Intervention/treatment variables (what we manipulate) |
| $\mathbf{V}$ | All observed variables in the system |
| $\mathbf{J}$ | Background intervention variables (fixed experimental conditions) |
| $P(\mathbf{V} \| do(\mathbf{J}))$ | Observational distribution under background interventions |
| $do(\cdot)$ | Intervention operator  |

---

#### 2. Definitions

**Function signature:** $\mathbf{ID}: (G, \mathbf{Y}, \mathbf{W}, P(\mathbf{V} | do(\mathbf{J}))) \rightarrow P(\mathbf{Y} | do(\mathbf{J}, \mathbf{W}))$ or FAIL

**Input constraints (from Line 2 of Algorithm 1):**
- $\mathbf{Y} \subseteq \mathbf{V}$ (target variables must be observed)
- $\mathbf{W} \subseteq \mathbf{V}$ (intervention variables must be observed)
- $\mathbf{Y} \cap \mathbf{W} = \emptyset$ (cannot intervene on outcome variables)

---

#### 3. English Explanation

This function answers: **"Can we predict the effect of intervention $do(\mathbf{W})$ on outcome $\mathbf{Y}$ using only observational data?"**

**Setup:**
- We have a causal graph $G$ describing relationships (including cycles and confounders)
- We have observational data: $P(\mathbf{V} | do(\mathbf{J}))$ collected under fixed conditions $\mathbf{J}$
- We want to know: what would happen to $\mathbf{Y}$ if we intervened on $\mathbf{W}$?

**The Goal:**
Determine if $P(\mathbf{Y} | do(\mathbf{J}, \mathbf{W}))$ can be expressed using only $P(\mathbf{V} | do(\mathbf{J}))$, without actually performing the intervention on $\mathbf{W}$.

---


### Line 2 - Precondition Check

$$
\textbf{2: require: } \mathbf{Y} \subseteq \mathbf{V}, \mathbf{W} \subseteq \mathbf{V}, \mathbf{Y} \cap \mathbf{W} = \emptyset
$$

---

#### 1. Symbols

| Symbol | Meaning |
|--------|---------|
| $\textbf{require}$ | Precondition that must be satisfied before proceeding |
| $\mathbf{Y}$ | Target/outcome variables |
| $\mathbf{W}$ | Intervention/treatment variables |
| $\mathbf{V}$ | All observed variables |
| $\subseteq$ | Subset relation (contained in) |
| $\cap$ | Set intersection |
| $\emptyset$ | Empty set |

---

#### 2. Definitions

**Three preconditions must hold:**

1. **$\mathbf{Y} \subseteq \mathbf{V}$**: Target variables are observed
   - Formally: $\forall y \in \mathbf{Y}, y \in \mathbf{V}$

2. **$\mathbf{W} \subseteq \mathbf{V}$**: Intervention variables are observed
   - Formally: $\forall w \in \mathbf{W}, w \in \mathbf{V}$

3. **$\mathbf{Y} \cap \mathbf{W} = \emptyset$**: Target and intervention sets are disjoint
   - Formally: $\nexists x : x \in \mathbf{Y} \land x \in \mathbf{W}$

---

#### 3. English Explanation

This line checks three conditions that must all be true before the algorithm can proceed:

**Condition 1: $\mathbf{Y} \subseteq \mathbf{V}$ - "Target variables must be observable"**
- Cannot identify causal effects on variables we don't measure
- Every variable we want to predict must be in our dataset

**Condition 2: $\mathbf{W} \subseteq \mathbf{V}$ - "Intervention variables must be observable"**
- Cannot intervene on variables we cannot observe or control
- Must be able to measure variables we're manipulating to verify the intervention

**Condition 3: $\mathbf{Y} \cap \mathbf{W} = \emptyset$ - "Target and intervention sets cannot overlap"**
- Prevents circular questions like "What is the effect of X on X?"
- If we're fixing X to a specific value, we already know its value
- Cannot simultaneously intervene on X and ask what happens to X

**If any condition fails:** The algorithm immediately stops and cannot proceed.

---

#### 4. Assumptions

**No assumptions for this line.**

These are **preconditions** that are verified by the `require:` statement. If violated, the algorithm terminates with an error. They define when the identification problem is well-posed, not assumptions about the model or data.



### Line 3 - Ancestral Closure

$$
\textbf{3: } H \leftarrow \text{Anc}^{G_{\mathbf{V} \setminus \mathbf{W}}}(\mathbf{Y})
$$

---

#### 1. Symbols

| Symbol | Meaning |
|--------|---------|
| $H$ | Set of relevant variables (result of this operation) |
| $\leftarrow$ | Assignment operator |
| $\text{Anc}^G(\mathbf{Y})$ | Ancestors of $\mathbf{Y}$ in graph $G$ |
| $G_{\mathbf{V} \setminus \mathbf{W}}$ | Subgraph induced by nodes $\mathbf{V} \setminus \mathbf{W}$ |
| $\mathbf{V} \setminus \mathbf{W}$ | Set difference: all variables in $\mathbf{V}$ except those in $\mathbf{W}$ |

---

#### 2. Definitions

**Ancestor set (Definition from paper, Section 2.1):**
$$\text{Anc}^G(\mathbf{Y}) = \{v \in \mathbf{V} : \exists \text{ directed path } v \rightsquigarrow y \text{ in } G \text{ for some } y \in \mathbf{Y}\} \cup \mathbf{Y}$$

**Induced subgraph $G_{\mathbf{V} \setminus \mathbf{W}}$:**
- Graph containing only nodes in $\mathbf{V} \setminus \mathbf{W}$
- Retains all edges between remaining nodes from original graph $G$

**Result:** 
$$H = \text{Anc}^{G_{\mathbf{V} \setminus \mathbf{W}}}(\mathbf{Y})$$

**Properties of $H$:**
- $\mathbf{Y} \subseteq H$ (always includes target variables)
- $H \cap \mathbf{W} = \emptyset$ (intervention variables excluded)
- $H \subseteq \mathbf{V} \setminus \mathbf{W}$ (subset of non-intervention variables)

---

#### 3. English Explanation

This line identifies the **causally relevant variables** for predicting $\mathbf{Y}$ under intervention $do(\mathbf{W})$.

**Two-step process:**

**Step 1: Remove intervention nodes**
- Create modified graph $G_{\mathbf{V} \setminus \mathbf{W}}$ by removing all nodes in $\mathbf{W}$
- Represents the semantics of $do(\mathbf{W})$: "cut off what naturally causes $\mathbf{W}$"
- After intervention, $\mathbf{W}$ is determined by the experimenter, not by the causal system

**Step 2: Find ancestors of $\mathbf{Y}$**
- In the modified graph, find all variables with directed paths to any variable in $\mathbf{Y}$
- These are the only variables that can causally influence $\mathbf{Y}$ (after intervening on $\mathbf{W}$)
- Variables with no path to $\mathbf{Y}$ are irrelevant for identification

**Why this matters:**
- Reduces the identification problem to a smaller set $H$ instead of all variables $\mathbf{V}$
- Focuses computation on causally relevant variables only
- Forms the basis for the decomposition in subsequent lines

---

#### 4. Assumptions

**Ancestral Closure Principle**

**From Lemma 9.7 (page 8):** For ancestral set $A$ where $\text{Anc}^G(A) = A$:

$$P_{M[A]}(A \cap \mathbf{V} | do(A \cap \mathbf{J})) = P_M(A \cap \mathbf{V} | do(\mathbf{J} \cup \mathbf{W}))$$

**What this assumes for Line 3:**
- The ancestral set $H = \text{Anc}^{G_{\mathbf{V} \setminus \mathbf{W}}}(\mathbf{Y})$ forms a valid sub-ioSCM (Definition 9.6, page 8)
- Variables outside $H$ are causally irrelevant for computing $P(\mathbf{Y} | do(\mathbf{J}, \mathbf{W}))$

**From Definition 2.10 (page 3):** Removing nodes $\mathbf{W}$ from graph $G$ corresponds to intervention $do(\mathbf{W})$ (graph surgery = intervention)

**From page 9:** Algorithm exploits that causal effects onto ancestral subsets are identifiable

### Line 4 - Loop Over Consolidated Districts

$$
\textbf{4: } \textbf{for } C \in \text{CD}(H) \textbf{ do}
$$

---

#### 1. Symbols

| Symbol | Meaning |
|--------|---------|
| $\textbf{for}$ ... $\textbf{do}$ | Loop control structure |
| $C$ | A consolidated district (subset of variables in $H$) |
| $\in$ | Set membership |
| $\text{CD}(H)$ | Set of all consolidated districts in subgraph $H$ |
| $H$ | Ancestral closure from Line 3 |

---
#### 2. Definitions

**Consolidated District (Definition 9.1, page 8):**

Let $G$ be a directed mixed graph (DMG) with set of nodes $\mathbf{V}$. For $v \in \mathbf{V}$, the consolidated district $\text{Cd}^G(v)$ of $v$ in $G$ is:

$$\text{Cd}^G(v) = \{w \in \mathbf{V} : \exists k \geq 1 \text{ nodes } (v_1, \ldots, v_k) \text{ where } v_1 = v, v_k = w$$
$$\text{ and } \forall i=2,\ldots,k: v_{i-1} \leftrightarrow v_i \in G \text{ or } v_i \in \text{Sc}^G(v_{i-1})\}$$

**In words:** 
- $w$ is in the consolidated district of $v$ if you can reach $w$ from $v$ through a sequence where each step is either:
  1. A bidirected edge $\leftrightarrow$ (latent confounder), OR
  2. Being in the same strongly connected component (feedback loop)

**Set of consolidated districts:**
$$\text{CD}(G) = \{\text{Cd}^G(v) : v \in \mathbf{V}\}$$

**Properties:**
- Consolidated districts partition $\mathbf{V}$ (every variable is in exactly one district)
- In acyclic graphs without bidirected edges: $\text{CD}(G) = \{\{v\} : v \in \mathbf{V}\}$ (each variable is its own district)
- With cycles or confounders: districts can contain multiple variables

---

#### 3. English Explanation

This line begins a loop that processes each **consolidated district** in $H$ separately.

**What is a consolidated district?**

A maximal set of variables coupled together by:
1. **Latent confounders** (bidirected edges $\leftrightarrow$), OR
2. **Feedback loops** (same strongly connected component)

**Why loop over districts?**
- Variables within a district must be identified together (cannot be separated)
- Different districts can be processed independently
- Divide-and-conquer strategy: solve one district at a time

**In acyclic graphs without confounders:** Each district = one variable

**In cyclic graphs or with confounders:** Districts can contain multiple variables


---

#### 4. Assumptions

**Decomposition via Consolidated Districts**

**From Proposition 9.8, Point 3 (page 8):**

For $D \subseteq \mathbf{V}$ a consolidated district of $G$:

$$P(D | do(\mathbf{J} \cup \mathbf{V} \setminus D)) = \bigotimes_{S \in \mathcal{S}(G), S \subseteq D} P(S | \text{Pred}^G_<(S) \cap \mathbf{V}, do(\mathbf{J}))$$

**What this means:**
- The causal effect onto a consolidated district $D$ can be decomposed into a product over its strongly connected components
- Each consolidated district can be identified as an independent subproblem
- This justifies processing districts separately in the loop

**From Definition 9.1 (page 8) - Consolidated Districts:**
- Consolidated districts generalize the notion of C-components/districts from acyclic graphs (ADMGs) to general directed mixed graphs (DMGs) with cycles
- In acyclic case: consolidated districts reduce to standard C-components

**From page 9 (Algorithm description):**
- "The main idea of the generalized ID algorithm is to exploit that the causal effects onto ancestral subsets and consolidated districts are identifiable"
- Line 4 operationalizes the "consolidated districts" part of this principle

**Key structural assumption:**
- The graph structure $G$ allows factorization over consolidated districts
- Causal effects onto consolidated districts are identifiable subproblems (proven in Proposition 9.8)

---

### Line 5 - Calling IDCD for each District

$$
\textbf{5: } Q[C] \leftarrow \text{IDCD}(G, C, \text{Cd}^G(C), Q[\text{Cd}^G(C)])
$$
---
#### 1. Symbols

| Symbol | Meaning |
|--------|---------|
| $Q[C]$ | Identified distribution for district $C$ (output) |
| $\leftarrow$ | Assignment operator |
| $\text{IDCD}$ | Helper function for identifying consolidated districts (defined in Line 13) |
| $G$ | Original causal graph |
| $C$ | Current consolidated district being processed |
| $\text{Cd}^G(C)$ | Consolidated district containing $C$ in graph $G$ |
| $Q[\text{Cd}^G(C)]$ | Previously computed distribution for $\text{Cd}^G(C)$ |

---

#### 2. Definitions

**Function call:** $\text{IDCD}(G, C, D, Q[D])$ where:
- $G$ = causal graph
- $C$ = target set (variables to identify)
- $D$ = consolidated district containing $C$ (where $\text{CD}(G_D) = \{D\}$)
- $Q[D]$ = distribution over $D$

**From Algorithm 1, Line 13:**
```
function IDCD(G, C, D, Q[D])
    require: C ⊆ D ⊆ V, CD(G_D) = {D}
```

**Input to IDCD for Line 5:**
- Target set: $C$ (the current district from the loop)
- Consolidated district: $\text{Cd}^G(C)$ (which equals $C$ in this context)
- Distribution: $Q[\text{Cd}^G(C)]$ (the distribution to work with)

**Output:** 
- $Q[C]$: The identified distribution $P(C | \text{Pred}^G_<(C) \cap H, do(\mathbf{J}))$

---

#### 3. English Explanation

This line calls the helper function **IDCD** to attempt identification of the current consolidated district $C$.

**What IDCD does:**
- Takes a consolidated district $C$ and attempts to identify its causal effect
- Uses recursive decomposition (alternating between ancestral closure and district decomposition)
- Returns either an identified distribution $Q[C]$ or FAIL

**The inputs:**
1. $G$ - the full causal graph (for structural information)
2. $C$ - the district we're trying to identify
3. $\text{Cd}^G(C)$ - the consolidated district containing $C$ (equals $C$ here since $C \in \text{CD}(H)$)
4. $Q[\text{Cd}^G(C)]$ - the distribution over that district

**Why this matters:**
- Each district must be identified separately
- IDCD handles the complexity of cycles and confounders within the district
- If IDCD returns FAIL for any district, the entire algorithm fails (checked in Line 6)

---

#### 4. Assumptions

**Input Requirements:**
- $G$ is the induced DMG of the ioSCM (Definition 5.1, page 5)
- $C$ is a valid consolidated district in $H$ (Definition 9.1, page 8)
- $Q[D]$ is a known distribution over $D = \text{Cd}^G(C)$

---

**Mathematical Basis:**

**Proposition 9.8, Point 2 (page 8):** Causal effects on consolidated districts in $M$ equal effects in sub-ioSCM $M[D]$, justifying the recursive call to IDCD

**Lemma 9.7 (page 8):** Causal effects onto ancestral subsets are identifiable, ensuring the ancestral closure step in IDCD is valid

**Remark 9.3 (page 8):** Apt-orders always exist for any DMG, required for decomposition in IDCD

---

**Statistical Assumptions:**

**Theorem 9.10 (page 9):** For each strongly connected component $S \subseteq \mathbf{V}$, a measure $\mu_S$ exists such that $P(\mathbf{V} | do(\mathbf{J}))$ has a density w.r.t. $\bigotimes_{S} \mu_S$, ensuring factorization is well-defined

---

**Algorithmic Strategy:**

**Remark 9.11 (page 9):** Algorithm alternates between ancestral closure and consolidated district decomposition to recursively shrink toward $C$ until either identified or proven non-identifiable

---

### Lines 6-9 - Checking for Identification Failure, Error handling, and Loop Completion


$$
\begin{align*}
\textbf{6: } & \textbf{if } Q[C] = \text{FAIL} \textbf{ then} \\
\textbf{7: } & \quad \textbf{return } \text{FAIL} \\
\textbf{8: } & \textbf{end if} \\
\textbf{9: } & \textbf{end for}
\end{align*}
$$

---

#### 1. Symbols

| Symbol | Meaning |
|--------|---------|
| $\textbf{if}$ ... $\textbf{then}$ | Conditional control structure |
| $Q[C]$ | Result from IDCD call (Line 5) |
| $\text{FAIL}$ | Special return value indicating non-identifiability |
| $\textbf{return}$ | Exit function and return value |
| $\textbf{end if}$ | Close conditional block |
| $\textbf{end for}$ | Close loop from Line 4 |

---
#### 2. Definitions

**Control flow:**

**Line 6:** Check if identification of district $C$ failed
- If `Q[C] = FAIL` → Execute Line 7
- If `Q[C] ≠ FAIL` → Skip to Line 9

**Line 7:** Terminate entire algorithm with failure
- Return `FAIL` immediately
- Stop processing remaining districts
- Algorithm exits here

**Line 8:** End of conditional block
- Closes the `if` statement from Line 6

**Line 9:** End of district loop
- Closes the `for` loop from Line 4
- Only reached if all districts were successfully identified
- Execution continues to Line 10

**Execution paths:**
1. **Any district fails:** Lines 6 → 7 (return FAIL, algorithm terminates)
2. **All districts succeed:** Lines 6 → 9 → 10 (continue to combine results)

---

#### 3. English Explanation

These lines handle success/failure of the district identification loop.

**The logic:**

**Line 6:** After attempting to identify district $C$ (Line 5), check if it failed

**Line 7:** If any district fails identification:
- Stop immediately (early termination)
- Return `FAIL` to caller
- Don't waste time processing remaining districts

**Lines 8-9:** Close the control structures:
- End the `if` block (Line 8)
- End the `for` loop (Line 9)

**Two possible outcomes after Line 9:**

**Scenario 1: At least one district failed**
- Algorithm terminated at Line 7
- Never reached Line 9
- Returns `FAIL`

**Scenario 2: All districts succeeded**
- Loop completed normally (Line 9)
- All `Q[C]` values contain valid distributions
- Continue to Line 10 to combine results

---

#### 4. Assumptions

**No additional assumptions for Lines 6-9.**

If a sub-problem cannot be identified using these techniques, the overall algorithm returns FAIL.